In [1]:
from pathlib import Path 
import IMP 
import IMP.atom

In [3]:
m = IMP.Model() 
h = IMP.atom.read_pdb(str(Path(Path.home(), "Documents/xray/data/pdbs/3ca7/3ca7.pdb")), m, IMP.atom.AllPDBSelector())

begin read_pdb:
  WARNING  Could not determine CHARMM atom type for atom "HET: O  " in residue #172 "HOH"
end read_pdb


In [5]:
pids = IMP.atom.Selection(h).get_selected_particle_indexes()
for pid in pids: 
    at = IMP.atom.Atom(m, pid)
    print(at.get_name())


Atom N of residue 48
Atom CA of residue 48
Atom C of residue 48
Atom O of residue 48
Atom CB of residue 48
Atom OG1 of residue 48
Atom CG2 of residue 48
Atom N of residue 49
Atom CA of residue 49
Atom C of residue 49
Atom O of residue 49
Atom CB of residue 49
Atom CG of residue 49
Atom CD1 of residue 49
Atom CD2 of residue 49
Atom CE1 of residue 49
Atom CE2 of residue 49
Atom CZ of residue 49
Atom N of residue 50
Atom CA of residue 50
Atom C of residue 50
Atom O of residue 50
Atom CB of residue 50
Atom CG of residue 50
Atom CD of residue 50
Atom N of residue 51
Atom CA of residue 51
Atom C of residue 51
Atom O of residue 51
Atom CB of residue 51
Atom OG1 of residue 51
Atom CG2 of residue 51
Atom N of residue 52
Atom CA of residue 52
Atom C of residue 52
Atom O of residue 52
Atom CB of residue 52
Atom CG of residue 52
Atom CD1 of residue 52
Atom CD2 of residue 52
Atom CE1 of residue 52
Atom CE2 of residue 52
Atom CZ of residue 52
Atom OH of residue 52
Atom N of residue 53
Atom CA of res